In [1]:
#import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable


In [2]:
import glob
import random
import os
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torchvision
from torch.utils.data import Dataset
from PIL import Image


In [3]:
#!pip install visdom


In [4]:
import visdom
import numpy as np
from pathlib import Path

In [5]:
from data import *
from models import *
from utils import *
import math


In [6]:
#TRAIN OPTIONS FROM GATED GAN
epoch = 0
n_epochs = 10
decay_epoch = 5
batchSize = 1
dataroot = './imslp'
loadSize = 300
fineSize = 256
ngf = 64
ndf = 64    
in_nc = 3 
out_nc = 3 
lr = 0.0002 
gpu = 1 
lambda_A = 10.0
pool_size = 50
resize_or_crop = 'resize_and_crop'
autoencoder_constrain = 10 
n_styles = 4
cuda=False
tv_strength=1e-6


In [7]:
dataloader = DataLoader(ImageDataset(dataroot), 
                        batch_size=1, shuffle=True, num_workers=4)

batch = next(iter(dataloader))


Style: ./imslp/trainStyles/ili
Style: ./imslp/trainStyles/yellowed


In [8]:
batch['style_label']

tensor([1])

In [9]:
generator = Generator(in_nc, out_nc, n_styles, ngf)
discriminator = Discriminator(in_nc, n_styles, ndf)

In [10]:
#generator.load_state_dict(torch.load('./output/netG.pth'))

In [11]:
if cuda:
    generator.cuda()
    discriminator.cuda()

In [12]:
#Losses Init
use_lsgan=True
if use_lsgan:
    criterion_GAN = nn.MSELoss()
else: 
    criterion_GAN = nn.BCELoss()
    
    
criterion_ACGAN = nn.CrossEntropyLoss()
criterion_Rec = nn.L1Loss()
criterion_TV = TVLoss(TVLoss_weight=tv_strength)




In [13]:
#Optimizers & LR schedulers
optimizer_G = torch.optim.Adam(generator.parameters(),
                                lr=lr, betas=(0.5, 0.999))
optimizer_D = torch.optim.Adam(discriminator.parameters(), 
                               lr=lr, betas=(0.5, 0.999))


lr_scheduler_G = torch.optim.lr_scheduler.LambdaLR(optimizer_G, lr_lambda=LambdaLR(n_epochs, epoch,decay_epoch).step)
lr_scheduler_D = torch.optim.lr_scheduler.LambdaLR(optimizer_D, lr_lambda=LambdaLR(n_epochs,epoch, decay_epoch).step)



In [14]:
#Set vars for training
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
input_A = Tensor(batchSize, in_nc, fineSize, fineSize)
input_B = Tensor(batchSize, out_nc, fineSize, fineSize)
target_real = Variable(Tensor(batchSize).fill_(1.0), requires_grad=False)
target_fake = Variable(Tensor(batchSize).fill_(0.0), requires_grad=False)

D_A_size = discriminator(input_A.copy_(batch['style']))[0].size()  
D_AC_size = discriminator(input_B.copy_(batch['style']))[1].size()

class_label_B = Tensor(D_AC_size[0],D_AC_size[1],D_AC_size[2]).long()

autoflag_OHE = Tensor(1,n_styles+1).fill_(0).long()
autoflag_OHE[0][-1] = 1

fake_label = Tensor(D_A_size).fill_(0.0)
real_label = Tensor(D_A_size).fill_(0.99) 

rec_A_AE = Tensor(batchSize,in_nc,fineSize,fineSize)

fake_buffer = ReplayBuffer()

##Init Weights
generator.apply(weights_init_normal)


Generator(
  (encoder): Encoder(
    (model): Sequential(
      (0): ReflectionPad2d((3, 3, 3, 3))
      (1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1))
      (2): InstanceNorm2d(64, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (3): ReLU(inplace=True)
      (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (5): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (8): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
      (9): ReLU(inplace=True)
    )
  )
  (transformer): Transformer(
    (t): ModuleList(
      (0): ResidualBlock(
        (conv_block): Sequential(
          (0): ReflectionPad2d((1, 1, 1, 1))
          (1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1))
          (2): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=Fals

In [15]:
discriminator.apply(weights_init_normal)

Discriminator(
  (model): Sequential(
    (0): Conv2d(3, 64, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (3): InstanceNorm2d(128, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (4): LeakyReLU(negative_slope=0.2, inplace=True)
    (5): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(2, 2))
    (6): InstanceNorm2d(256, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (7): LeakyReLU(negative_slope=0.2, inplace=True)
    (8): Conv2d(256, 512, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
    (9): InstanceNorm2d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (10): LeakyReLU(negative_slope=0.2, inplace=True)
  )
  (fldiscriminator): Conv2d(512, 1, kernel_size=(4, 4), stride=(1, 1), padding=(2, 2))
  (aux_clf): Conv2d(512, 4, kernel_size=(4, 4), stride=(1, 1), paddin

In [16]:
logger = Logger(n_epochs, len(dataloader))


Setting up a new session...
Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/urllib3/connection.py", line 169, in _new_conn
    conn = connection.create_connection(
  File "/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/urllib3/util/connection.py", line 96, in create_connection
    raise err
  File "/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/urllib3/util/connection.py", line 86, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/urllib3/connectionpool.py", line 699, in urlopen
    httplib_response = self._make_request(
  File "/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/urllib3/connectionpool.py", line 394, in _ma

Exception in user code:
------------------------------------------------------------


In [17]:
# #source image
# real_content = Variable(input_A.copy_(batch['content']))
# #target style
# real_style = Variable(input_B.copy_(batch['style']))
# #simple label of style index
# style_label = batch['style_label']
# #one hot encoded style label
# style_OHE = F.one_hot(style_label,n_styles).long()
# #Style Label mapped over 1x19x19 tensor for discriminator 
# class_label = class_label_B.copy_(label2tensor(style_label,class_label_B)).long()


In [ ]:
from datetime import datetime
Path("output").mkdir(exist_ok=True)
### TRAIN LOOP
for epoch in range(epoch, n_epochs):
    print(f"EPOCH {epoch} ({datetime.now()})")
    for i, batch in enumerate(dataloader):
        ## Unpack minibatch
        # source content
        real_content = Variable(input_A.copy_(batch['content']))
        # target style
        real_style = Variable(input_B.copy_(batch['style']))
        # style label
        style_label = batch['style_label']
        # one-hot encoded style
        style_OHE = F.one_hot(style_label,n_styles).long()
        # style Label mapped over 1x19x19 tensor for patch discriminator 
        class_label = class_label_B.copy_(label2tensor(style_label,class_label_B)).long()
        
        #### Update Discriminator
        optimizer_D.zero_grad()
        
        # Generate style-transfered image
        genfake = generator({
            'content':real_content,
            'style_label': style_OHE})
        
        # Add generated image to image pool and randomly sample pool 
        fake = fake_buffer.push_and_pop(genfake)
        # Discriminator forward pass with sampled fake 
        out_gan, out_class = discriminator(fake)
        # Discriminator Fake loss (correctly identify generated images)
        errD_fake = criterion_GAN(out_gan, fake_label)
        # Backward pass and parameter optimization
        errD_fake.backward()
        optimizer_D.step()
        
        optimizer_D.zero_grad()
        # Discriminator forward pass with target style
        out_gan, out_class = discriminator(real_style)
        # Discriminator Style Classification loss
        errD_real_class = criterion_ACGAN(out_class.transpose(1,3),class_label)*lambda_A
        # Discriminator Real loss (correctly identify real style images)
        errD_real = criterion_GAN(out_gan, real_label)        
        errD_real_total = errD_real + errD_real_class
        # Backward pass and parameter optimization
        errD_real_total.backward()
        optimizer_D.step()
        
        
        errD = (errD_real+errD_fake)/2.0
        
                
        #### Generator Update
        ## Style Transfer Loss
        optimizer_G.zero_grad()
        
        # Discriminator forward pass with generated style transfer
        out_gan, out_class = discriminator(genfake)
        
        # Generator gan (real/fake) loss
        err_gan = criterion_GAN(out_gan, real_label)
        # Generator style class loss
        err_class = criterion_ACGAN(out_class.transpose(1,3), class_label)*lambda_A
        # Total Variation loss
        err_TV = criterion_TV(genfake)
        
        errG_tot = err_gan + err_class + err_TV 
        
        errG_tot.backward()
        optimizer_G.step()
        
        ## Auto-Encoder (Recreation) Loss
        optimizer_G.zero_grad()
        identity = generator({
            'content': real_content,
            'style_label': autoflag_OHE,
        })
        err_ae = criterion_Rec(identity,real_content)*autoencoder_constrain
        err_ae.backward()
        optimizer_G.step()
        
        
        #Progress report (port 8097)
#         logger.log({'Generator Loss': errG_tot,
#                     'Reconstruction Loss': err_ae,
#                     'loss_G_GAN': err_gan,
#                     'loss_G_AC': err_class,
#                     'Discriminator GAN Loss': errD,
#                     'tv_loss':err_TV,
#                     'Discriminator Class Loss': errD_real_class}, 
#                     images={'content': real_content,
#                             'style': real_style,
#                             'transfer': genfake,
#                             'auto-reconstruction':identity})
        if i % 50 == 0:
            print(
                f"LossGEN: {float(errG_tot):.5}, " +
                f"LossRec: {float(err_ae):.5}, " +
                f"loss_G_GAN: {float(err_gan):.5}, " +
                f"loss_G_AC: {float(err_class):.5}, " +
                f"LossDESCR GAN: {float(errD):.5}, " +
                f"tv_loss: {float(err_TV):.5}, " +
                f"LossDESCR Class: {float(errD_real_class):.5}"
            )

    ##update learning rates
    lr_scheduler_G.step()
    lr_scheduler_D.step()
    
    #Save model
    torch.save(generator.state_dict(), f'output/netG_{epoch}.pth')
    torch.save(discriminator.state_dict(), f'output/netD_{epoch}.pth')
    print("=" * 20)

        


EPOCH 0 (2021-08-17 15:42:30.481626)
LossGEN: 9.1278, LossRec: 7.4899, loss_G_GAN: 2.8651, loss_G_AC: 6.0943, LossDESCR GAN: 1.6749, tv_loss: 0.16844, LossDESCR Class: 10.656


/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (99313084 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (157393683 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (97410768 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (157218750 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/py

LossGEN: 2.29, LossRec: 1.4286, loss_G_GAN: 0.83485, loss_G_AC: 1.4285, LossDESCR GAN: 0.28037, tv_loss: 0.026601, LossDESCR Class: 2.7543


/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (96355168 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (145942116 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (157552000 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (97913365 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/py

LossGEN: 5.7173, LossRec: 0.8297, loss_G_GAN: 0.33413, loss_G_AC: 5.3621, LossDESCR GAN: 0.28937, tv_loss: 0.021027, LossDESCR Class: 23.063


...

LossGEN: 1.1266, LossRec: 0.92258, loss_G_GAN: 1.0704, loss_G_AC: 0.037405, LossDESCR GAN: 0.088055, tv_loss: 0.018786, LossDESCR Class: 0.014384


/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (157359384 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (103960356 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (157348938 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (97326320 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/p

LossGEN: 0.71914, LossRec: 0.94019, loss_G_GAN: 0.60316, loss_G_AC: 0.099838, LossDESCR GAN: 0.19999, tv_loss: 0.016142, LossDESCR Class: 0.030693


/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (157541130 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (99683804 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(
/home/ubuntu/miniconda3/envs/style_transfer/lib/python3.8/site-packages/PIL/Image.py:2911: DecompressionBombWarning: Image size (97357988 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


LossGEN: 0.17954, LossRec: 0.88115, loss_G_GAN: 0.10725, loss_G_AC: 0.038727, LossDESCR GAN: 0.071498, tv_loss: 0.033565, LossDESCR Class: 0.02586
